In [1]:
# 25 nm - 1550 nm

import os
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import imp
import sys
import pickle as plk
import importlib.util
from numpy.lib.scimath import sqrt as csqrt
from IPython.display import display, Math, clear_output
from scipy.signal import find_peaks
###############################################################
#LINUX PATH

# sys.path.append("/opt/lumerical/v221/api/python") #Default linux lumapi path
# import lumapi
# sys.path.append(os.path.dirname('/media/leonardo/datam02/Ring Resonator - Leonardo Pessôa/Ring-Resonator/Tutorial/')) #Current directory

###############################################################
# WINDOWS PATH
spec_win = importlib.util.spec_from_file_location('lumapi', 'C:\\Program Files\\Lumerical\\v202\\api\\python\\lumapi.py')
lumapi = importlib.util.module_from_spec(spec_win)
os.add_dll_directory("C:/Program Files/Lumerical/v202/api/python")
sys.path.append(os.path.dirname('D:/AAAAAAAArquivos/Facul/Capacitação/GitHub/BraggGrating'))
spec_win.loader.exec_module(lumapi)

def fwhm(band, reflection, dB):
    from scipy.signal import find_peaks
    peak, pd = find_peaks(reflection)
    rmax = reflection[peak[np.argmax(reflection[peak])]]

    min = 0
    max = 0

    if dB == True: 
        dB = 3
        rmax = rmax-dB
    else: 
        dB = 0 
        rmax = rmax/2

    for i in range(peak[np.argmax(reflection[peak])], 0, -1):
        if (reflection[i] <(rmax) and reflection[i+1] < (rmax)):
            min = i
            break

    for i in range(peak[np.argmax(reflection[peak])], len(band) - 1):
        if (reflection[i-1] > (rmax) and reflection[i] < (rmax)):
            max = i
            break

    print('FWHM = '+ str(np.round((band[max]-band[min])*1e9,4)) + 'nm')

    r = (reflection[min] +reflection[max])/2

    
    return r,min,max

C:\Users\lecob\AppData\Local\Temp\ipykernel_10728\57351922.py:8: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [2]:
fdtd = lumapi.FDTD()

In [4]:
# Simulation on EME

dw = 24e-9
N = 82
gratingPeriod = 344.3777e-9
aux = 0

fdtd.switchtolayout()
fdtd.deleteall()

fdtd.addrect()
fdtd.set({
    'x min'    : -2e-6,
    'x max'    : 0,
    'y'        : 0,
    'y span'   : 0.45e-6,
    'z'        : 0.11e-6,
    'z span'   : 0.22e-6,
    'material' : 'Si (Silicon) - Palik',
    'name'     : 'waveguide'
})
for i in range(N):
    fdtd.unselectall()
    fdtd.addrect()
    fdtd.set({
        'x max'    : gratingPeriod/2 + aux,
        'x min'    : 0 + aux,
        'y'        : 0,
        'y span'   : 0.45e-6 + 2*dw,
        'z'        : 0.11e-6,
        'z span'   : 0.22e-6,
        'material' : 'Si (Silicon) - Palik',
        'name'     : 'grating' + str(i) + 'bigger'
    })
    fdtd.unselectall()
    fdtd.addrect()
    fdtd.set({
        'x max'    : gratingPeriod + aux,
        'x min'    : gratingPeriod/2 + aux,
        'y'        : 0,
        'y span'   : 0.45e-6 - 2*dw,
        'z'        : 0.11e-6,
        'z span'   : 0.22e-6,
        'material' : 'Si (Silicon) - Palik',
        'name'     : 'grating' + str(i) + 'smaller'
    })
    aux = aux+gratingPeriod

fdtd.unselectall()
fdtd.addrect()
fdtd.set({
    'x max'    : 2e-6 + aux,
    'x min'    : aux,
    'y'        : 0,
    'y span'   : 0.45e-6,
    'z'        : 0.11e-6,
    'z span'   : 0.22e-6,
    'material' : 'Si (Silicon) - Palik',
    'name'     : 'waveguide2'
})

fdtd.unselectall()
fdtd.addrect()
fdtd.set({
    'x'        : aux/2,
    'x span'   : 1.25*aux,
    'y'        : 0,
    'y span'   : 5e-6,
    'z span'   : 2e-6,
    'z max'    : 0e-6,
    'material' : 'SiO2 (Glass) - Palik',
    'name'     : 'substract'
})
